<a href="https://colab.research.google.com/github/vethssvikas/Work-done-in-DS-internship/blob/main/Aspect_Based_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
import math
import tensorflow as tf


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import *
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, r2_score

In [ ]:
source_word2idx = {}
target_word2idx = {}
source_data = []
source_loc_data = []
target_data = []
target_label = []

final_list = []
max_length = 0


In [ ]:
def read_data(file):
    data_train_1 = pd.read_csv(file)
    return data_train_1

In [ ]:
def read_and_process_data(data_train_1, sourceWord2idx, targetWord2idx):
    global source_word2idx
    global target_word2idx
    source_word2idx = sourceWord2idx
    target_word2idx = targetWord2idx
    data_train_1.apply(prepare_data,axis = 1)
    return source_data, source_loc_data, target_data, target_label, max_length


In [ ]:
def custom_tokenize(text):
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(text)
    words = [word for word in tokens if word.isalnum()]
    return words

In [ ]:
def parse_data(data_train_1):
    data_train_1[' text'] = data_train_1[' text'].apply(lambda x: x.replace('[comma]',',').lower())
    data_train_1[' text'] = data_train_1[' text'].apply(custom_tokenize)
    data_train_1[' aspect_term'] = data_train_1[' aspect_term'].apply(lambda x: x.lower())
    data_train_1[' aspect_term'] = data_train_1[' aspect_term'].apply(custom_tokenize)
    data_train_1[' aspect_term'] = data_train_1[' aspect_term'].apply(lambda x:" ".join(x))
    return data_train_1

In [ ]:
def prepare_data(row):
    global max_length
    global source_word2idx
    global target_word2idx
    m = [source_word2idx[id] for id in row[' text']]
    if len(m) == 2602:
        print(row[' text'])
    if len(m) > max_length:
        max_length = len(m)
    source_data.append(m)
    t = [target_word2idx[row[' aspect_term']]]
    target_data.append(t)
    target_label.append(row[' class'])
    g = get_pos(row)
    datab = []
    datab.append(m)
    datab.append(g)
    datab.append(t)
    final_list.append(datab)

In [ ]:
def get_pos(row):
    index = []
    s_len = len(row[' text'])-1
    p = row[' text'].copy()
    aspects = row[' aspect_term'].split(' ')
    for aspect in aspects:
        try:
            if len(aspects)-1 > aspects.index(aspect):
                a_i = [i for i,val in enumerate(row[' text']) if val==aspect]
                try:
                    for a_id in a_i:
                        if row[' text'][a_id+1] != aspects[aspects.index(aspect)+1]:
                            a_i.remove(a_id)
                except:
                    pass
                index.extend(a_i[0])
            else:
                index.append(row[' text'].index(aspect))
            p[row[' text'].index(aspect)] = s_len
        except:
            pass
    try:
        for i in range(index[0]):
            p[i] = s_len - index[0] + i
        v = s_len
        for i in range(index[len(index)-1],len(p)):
            if i == index[len(index)-1]:
                p[i] = v
            else:    
                p[i] = v - 1
                v = v-1
            
    except Exception as e: 
        p = [0 for i in row[' text']]
    source_loc_data.append(p)
    return p

In [ ]:
def create_vocab(data_train_1):
    source_word2idx = {'<pad>': 0}
    target_word2idx = {}
    for words in data_train_1[' text']:
        for word in words:
            if word not in source_word2idx:
                source_word2idx[word] = len(source_word2idx)

    for words in data_train_1[' aspect_term']:
        if words not in target_word2idx:
            target_word2idx[words] = len(target_word2idx)
    return source_word2idx, target_word2idx

In [ ]:
train_file = '/content/gdrive/My Drive/data_2_train.csv'

data = read_data(train_file)
parsed_data = parse_data(data)
source_word2idx, target_word2idx = create_vocab(parsed_data)
train_data = read_and_process_data(parsed_data, source_word2idx, target_word2idx)

In [ ]:

data.iloc[:,[1,2]]

,text,aspect_term
0,"[but, the, staff, was, so, horrible, to, us]",staff
1,"[to, be, completely, fair, the, only, redeemin...",food
2,"[the, food, is, uniformly, exceptional, with, ...",food
3,"[the, food, is, uniformly, exceptional, with, ...",kitchen
4,"[the, food, is, uniformly, exceptional, with, ...",menu
...,...,...
3597,"[each, table, has, a, pot, of, boiling, water,...",pot of boiling water
3598,"[each, table, has, a, pot, of, boiling, water,...",meats
3599,"[each, table, has, a, pot, of, boiling, water,...",vegetables
3600,"[each, table, has, a, pot, of, boiling, water,...",rice


In [ ]:
x1_list, x2_list, x3_list = list(), list(), list()
example_list = list()

sen_max_len = train_data[4]
print("sen_max_len: ",sen_max_len)

for i in range(len(train_data[0])):
    x1, x2, x3 = list(), list(), list()
    
    x1.extend(train_data[0][i])
    if len(x1) < sen_max_len:
        for _ in range(sen_max_len - len(x1)):
            x1.append(0)
    
    x2.extend(train_data[1][i])
    if len(x2) < sen_max_len:
        for _ in range(sen_max_len - len(x2)):
            x2.append(0)
            
    x3.extend(train_data[2][i])
    for j in range(len(x1) - 1):
        x3.append(x3[0])
    for k in range(sen_max_len - len(x3)):
        x3.append(0)

    example = list()
    example.append(x1)
    example.append(x2)
    example.append(x3)
    example_list.append(example)



final_array = np.array(example_list)
nsamples, nx, ny = final_array.shape
feature_vector = final_array.reshape((nsamples,nx*ny))
labels = train_data[3] 



sen_max_len:  70


In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_sample(feature_vector, labels)


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [ ]:
train_vectors, test_vectors, train_labels, test_labels = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)

In [ ]:
print(train_vectors.shape)
print(train_labels.shape)
print(test_vectors.shape)
print(test_labels.shape)
print(type(train_labels))

(5193, 210)
(5193,)
(1299, 210)
(1299,)
<class 'numpy.ndarray'>


In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFR
n_estimators = [100,200,500]
rfr = RFR(n_estimators=100,random_state=0)

print("Results for RFR: ")
print()

rfr_scores = cross_val_score(rfr, train_vectors, train_labels, cv=10)  
print('scores: ', rfr_scores)

rfr.fit(train_vectors,train_labels)
y_pred = rfr.predict(test_vectors) 

rfr_accuracy = accuracy_score(test_labels, y_pred)

print()

print("")
print(classification_report(test_labels, y_pred))
print("Overall accuracy: ",rfr_accuracy)

Results for RFR: 

scores:  [0.74230769 0.73269231 0.77115385 0.78034682 0.78612717 0.8150289
 0.73988439 0.74566474 0.7938343  0.75337187]


              precision    recall  f1-score   support

          -1       0.78      0.77      0.77       433
           0       0.77      0.81      0.79       432
           1       0.76      0.74      0.75       434

    accuracy                           0.77      1299
   macro avg       0.77      0.77      0.77      1299
weighted avg       0.77      0.77      0.77      1299

Overall accuracy:  0.7713625866050808


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
print("Results for DecisionTreeClassifier: ")
print()
dt_scores = cross_val_score(dt_clf, train_vectors, train_labels, cv=10)  
print('scores: ', dt_scores)

dt_clf.fit(train_vectors,train_labels)
y_pred = dt_clf.predict(test_vectors)

accuracy = accuracy_score(test_labels, y_pred)

print("")
print(classification_report(test_labels, y_pred))
print("Overall accuracy: ",accuracy)

Results for DecisionTreeClassifier: 

scores:  [0.67115385 0.65576923 0.67115385 0.67244701 0.73025048 0.69749518
 0.65895954 0.64354528 0.65510597 0.65703276]

              precision    recall  f1-score   support

          -1       0.67      0.64      0.65       433
           0       0.65      0.72      0.68       432
           1       0.68      0.65      0.67       434

    accuracy                           0.67      1299
   macro avg       0.67      0.67      0.67      1299
weighted avg       0.67      0.67      0.67      1299

Overall accuracy:  0.6674364896073903


In [ ]:
from sklearn.ensemble import AdaBoostClassifier as abc
from sklearn.datasets import make_classification
dt_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
print("Results for AdaBoost Classifier: ")
print()
abc_scores = cross_val_score(dt_clf, train_vectors, train_labels, cv=10)  
print('scores: ', abc_scores)

dt_clf.fit(train_vectors,train_labels)
y_pred = dt_clf.predict(test_vectors)

accuracy = accuracy_score(test_labels, y_pred)

print("")
print(classification_report(test_labels, y_pred))
print("Overall accuracy: ",accuracy)

Results for AdaBoost Classifier: 

scores:  [0.52307692 0.55576923 0.54038462 0.522158   0.54527938 0.53564547
 0.54720617 0.52601156 0.58574181 0.52793834]

              precision    recall  f1-score   support

          -1       0.56      0.50      0.53       433
           0       0.54      0.57      0.55       432
           1       0.59      0.61      0.60       434

    accuracy                           0.56      1299
   macro avg       0.56      0.56      0.56      1299
weighted avg       0.56      0.56      0.56      1299

Overall accuracy:  0.5627405696689761


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(150,100,500), max_iter=3000,activation = 'relu',solver='adam',random_state=1)
print("Results for MLP Classifier: ")
print()
mlp_scores = cross_val_score(mlp, train_vectors, train_labels, cv=10)  
print('scores: ', mlp_scores)

dt_clf.fit(train_vectors,train_labels)
y_pred = dt_clf.predict(test_vectors)

accuracy = accuracy_score(test_labels, y_pred)

print("")
print(classification_report(test_labels, y_pred))
print("Overall accuracy: ",accuracy)

Results for MLP Classifier: 

scores:  [0.73653846 0.72307692 0.73461538 0.75529865 0.75722543 0.7495183
 0.68208092 0.72061657 0.74373796 0.74373796]

              precision    recall  f1-score   support

          -1       0.67      0.64      0.65       433
           0       0.65      0.72      0.68       432
           1       0.68      0.65      0.67       434

    accuracy                           0.67      1299
   macro avg       0.67      0.67      0.67      1299
weighted avg       0.67      0.67      0.67      1299

Overall accuracy:  0.6674364896073903
